In [2]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string


def col_header_val(df, table_config):
    """ replace whitespaces in the column  and standardized column names """
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file", mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Overwriting testutility.py


In [3]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    -Summons Number
    -Plate ID
    -Registration State
    -Plate Type
    -Issue Date
    -Violation Code
    -Vehicle Body Type
    -Vehicle Make
    -Issuing Agency
    -Street Code1
    -Street Code2
    -Street Code3
    -Vehicle Expiration Date
    -Violation Location
    -Violation Precinct
    -Issuer Precinct
    -Issuer Code
    -Issuer Command
    -Issuer Squad
    -Violation Time
    -Time First Observed
    -Violation County
    -Violation In Front Of Or Opposite
    -House Number
    -Street Name
    -Intersecting Street-Date First Observed
    -Law Section
    -Sub Division
    -Violation Legal Code
    -Days Parking In Effect    
    -From Hours In Effect
    -To Hours In Effect
    -Vehicle Color
    -Unregistered Vehicle?
    -Vehicle Year
    -Meter Number
    -Feet From Curb-Violation Post Code
    -Violation Description
    -No Standing or Stopping Violation
    -Hydrant Violation
    -Double Parking Violation
    -Latitude
    -Longitude
    -Community Board
    -Community Council 
    -Census Tract
    -BIN
    -BBL
    -NTA


Overwriting file.yaml


In [4]:
# reading config file
import pandas
import micropip
await micropip.install("pyyaml")
import testutility as util
config_data= util.read_config_file("file.yaml")

In [5]:
config_data['inbound_delimiter']

','

In [6]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': '-Summons Number -Plate ID -Registration State -Plate Type -Issue Date -Violation Code -Vehicle Body Type -Vehicle Make -Issuing Agency -Street Code1 -Street Code2 -Street Code3 -Vehicle Expiration Date -Violation Location -Violation Precinct -Issuer Precinct -Issuer Code -Issuer Command -Issuer Squad -Violation Time -Time First Observed -Violation County -Violation In Front Of Or Opposite -House Number -Street Name -Intersecting Street-Date First Observed -Law Section -Sub Division -Violation Legal Code -Days Parking In Effect -From Hours In Effect -To Hours In Effect -Vehicle Color -Unregistered Vehicle? -Vehicle Year -Meter Number -Feet From Curb-Violation Post Code -Violation Description -No Standing or Stopping Violation -Hydrant Violation -Double Parking Violation -Latitude -Longitude -Com

In [7]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("test_data.csv",delimiter=',')
df_sample.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()


<ipython-input-8-518d97d0919c>:5: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['street_code3', 'street_code2', 'street_code1', 'plate_type', 'summons_number', 'issuer_precinct', 'days_parking_in_effect', 'issue_date', 'bin', 'violation_time', 'from_hours_in_effect', 'census_tract', 'street_name', 'unregistered_vehicle', 'vehicle_expiration_date', 'violation_precinct', 'meter_number', 'violation_county', 'issuing_agency', 'hydrant_violation', 'latitude', 'longitude', 'vehicle_make', 'feet_from_curb', 'to_hours_in_effect', 'issuer_squad', 'violation_code', 'no_standing_or_stopping_violation', 'violation_description', 'violation_in_front_of_or_opposite', 'violation_post_code', 'house_number', 'date_first_observed', 'violation_legal_code', 'issuer_code', 'plate_id', 'registration_state', 'bbl', 'time_first_observed', 'double_parking_violation', 'community_board', 'sub_division', 'vehicle_color', 'vehicle_year', 'issuer_command', 'law_section', 'intersecting_street', 'vehi

0

In [10]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['summons_number', 'plate_id', 'registration_state', 'plate_type',
       'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make',
       'issuing_agency', 'street_code1', 'street_code2', 'street_code3',
       'vehicle_expiration_date', 'violation_location', 'violation_precinct',
       'issuer_precinct', 'issuer_code', 'issuer_command', 'issuer_squad',
       'violation_time', 'time_first_observed', 'violation_county',
       'violation_in_front_of_or_opposite', 'house_number', 'street_name',
       'intersecting_street', 'date_first_observed', 'law_section',
       'sub_division', 'violation_legal_code', 'days_parking_in_effect',
       'from_hours_in_effect', 'to_hours_in_effect', 'vehicle_color',
       'unregistered_vehicle', 'vehicle_year', 'meter_number',
       'feet_from_curb', 'violation_post_code', 'violation_description',
       'no_standing_or_stopping_violation', 'hydrant_violation',
       'double_parking_violation', 'latitude', '

In [11]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['street_code3', 'street_code2', 'street_code1', 'plate_type', 'summons_number', 'issuer_precinct', 'days_parking_in_effect', 'issue_date', 'bin', 'violation_time', 'from_hours_in_effect', 'census_tract', 'street_name', 'unregistered_vehicle', 'vehicle_expiration_date', 'violation_precinct', 'meter_number', 'violation_county', 'issuing_agency', 'hydrant_violation', 'latitude', 'longitude', 'vehicle_make', 'feet_from_curb', 'to_hours_in_effect', 'issuer_squad', 'violation_code', 'no_standing_or_stopping_violation', 'violation_description', 'violation_in_front_of_or_opposite', 'violation_post_code', 'house_number', 'date_first_observed', 'violation_legal_code', 'issuer_code', 'plate_id', 'registration_state', 'bbl', 'time_first_observed', 'double_parking_violation', 'community_board', 'sub_division', 'vehicle_color', 'vehicle_year', 'issuer_command', 'law_section', 'intersecting_street', 'vehi

In [12]:
pd.read_csv("test_data.csv")

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122742,7475935358,ETT3569,NY,PAS,12/28/2014,40,4DSD,TOYOT,T,47390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122743,8002415711,22822JK,NY,COM,07/23/2014,14,PICK,CHEVR,T,34570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122744,8003468346,GSY2804,NY,OMS,10/31/2014,38,4DSD,FORD,T,40890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122745,8030729595,GUW5439,NY,PAS,04/16/2015,21,4DSD,HONDA,T,13610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
